In [8]:
import requests
import json
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait

In [9]:
import pandas as pd
import numpy as np

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selectorlib import Extractor
import requests
import json
import time

## Loading pre-documented gender stereotyped items

In [4]:
stereo_toys = pd.read_csv('~/Desktop/IA CAPSTONE/predoc_stereotyped_items.csv', delimiter =',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


In [5]:
with open('../IA CAPSTONE/all_items.txt') as f:
    all_items = f.read().splitlines()

In [6]:
len(all_items)

166

In [7]:
trial = all_items[160:]
trial

['legos', 'scooter', 'drum set', 'puzzles', 'board games', 'rock painting']

In [8]:
generic = ['toys', 'books', 'learning material', 'games', 'sports']

In [9]:
gender = ['boys', 'girls', 'neutral']

## Search

### Getting asin of the retrieved products

In [10]:
def asin(driver):
    asin_list = []
    for index in range(1,10):
        asins = driver.find_elements("xpath","//div[@data-asin]")
        for asin in asins:
            #print(asin)
            asin_list.append(asin.get_attribute('data-asin'))
    return asin_list

### get information (title) of retirved products

In [11]:
from selenium.webdriver.common.by import By
def item_info(driver):
    item = []
    elem = driver.find_elements(By.CSS_SELECTOR,'h2')
    for i in elem:
        item.append(i.text)
    return item

### Getting product link of the retrievd products

In [12]:
def item_link(driver):
    href = []
    links = driver.find_elements('xpath',"//h2//a[@href]")
    for link in links:
        href.append(link.get_attribute('href'))
    return href

### Running queries for both boy and girls and neutral (kids)

In [13]:
def search(item, who):
    if who == 'neutral':
        query = item + ' for '+'kids'
    else:
        query = item + ' for '+who
    driver.get(f'https://www.amazon.co.uk/s?k={query}&ref=nb_sb_noss')
    driver.implicitly_wait(10)
    list_asin = asin(driver)
    item_list = item_info(driver)
    item_page = item_link(driver)
    return (list_asin, item_list), item_page

In [14]:
import time
import warnings
driver = webdriver.Chrome(ChromeDriverManager().install())
search_result = {}
for i in generic:
    gender_dict = {}
    for g in gender:
        gender_dict[g] = search(i, g)
        driver.implicitly_wait(5)
    search_result[i] = gender_dict
    time.sleep(1.5)
driver.close()

/var/folders/c5/zlstbbh10mngtwbv3lm2f7vw0000gn/T/ipykernel_71312/112412462.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
import pandas as pd
columns1 = ['gender', 'query', 'result']
qr = pd.DataFrame(columns = columns1)
columns2 = ['gender', 'query', 'href']
qr_link = pd.DataFrame(columns = columns2)
#qr_link

In [16]:
trial

['legos', 'scooter', 'drum set', 'puzzles', 'board games', 'rock painting']

In [17]:
len(trial)

6

## running queries in a loop

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
data1 = []
data2 = []
item = ''
for item in all_items:
    for g in gender:
        result, link = search(item, g)
        values1 = [g, item, result]
        values2 = [g, item, link]
        zipped1 = zip(columns1, values1)
        zipped2 = zip(columns2, values2)
        a_dictionary1 = dict(zipped1)
        a_dictionary2 = dict(zipped2)
        #print(a_dictionary)
        time.sleep(1.5)
        data1.append(a_dictionary1)
        data2.append(a_dictionary2)
driver.close()

In [19]:
qr = qr.append(data1, True)
qr

,gender,query,result
0,boys,vehicle toys,"([, B09N783Q3T, B09XQXTC74, B07WQTSPGF, B0B426..."
1,girls,vehicle toys,"([, B07WQTSPGF, B09XQXTC74, B08ZCRVGHQ, B0759M..."
2,neutral,vehicle toys,"([, B09N783Q3T, B09J8BRFSP, B08ZCRVGHQ, B09XQX..."
3,boys,sport,"([, B081GW29YC, B00I04FDCI, B09ZKX11BT, B06XFQ..."
4,girls,sport,"([, B089W2C2Z8, B01HEZMR6I, B0B8HBZCRL, B0B9WW..."
...,...,...,...
493,girls,board games,"([, B092STXL3T, B09H7J66Z2, B00000JICB, B0BLHW..."
494,neutral,board games,"([, B092STXL3T, B07B7KPTQG, B078S8D27R, B078TW..."
495,boys,rock painting,"([, B08XK67TRQ, B07Z2R7S5P, B08RDCZWC7, B087PW..."
496,girls,rock painting,"([, B08XK67TRQ, B07Z2R7S5P, B08RDCZWC7, B087PW..."


In [20]:
qr_link = qr_link.append(data2, True)
qr_link

,gender,query,href
0,boys,vehicle toys,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
1,girls,vehicle toys,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
2,neutral,vehicle toys,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
3,boys,sport,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
4,girls,sport,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
...,...,...,...
493,girls,board games,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
494,neutral,board games,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
495,boys,rock painting,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
496,girls,rock painting,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...


In [21]:
first20 = pd.DataFrame()

In [22]:
first20 = first20.append(qr, ignore_index =True)
# first20_link = first20_link.append(qr_link, ignore_index =True)

In [63]:
#i = 0
#for x in first20_link['href']:
    #print(len(x))
   # if len(x) == 0  :
    #    print(i)
    #    first20_link.loc[i]['href'] = np.nan
   # i+=1

In [64]:
# first20_link.isna().sum()

In [65]:
# first20_link.dropna(inplace=True)

In [66]:
#first20_link.loc[114]

In [24]:
first20

,gender,query,result
0,boys,vehicle toys,"([, B09N783Q3T, B09XQXTC74, B07WQTSPGF, B0B426..."
1,girls,vehicle toys,"([, B07WQTSPGF, B09XQXTC74, B08ZCRVGHQ, B0759M..."
2,neutral,vehicle toys,"([, B09N783Q3T, B09J8BRFSP, B08ZCRVGHQ, B09XQX..."
3,boys,sport,"([, B081GW29YC, B00I04FDCI, B09ZKX11BT, B06XFQ..."
4,girls,sport,"([, B089W2C2Z8, B01HEZMR6I, B0B8HBZCRL, B0B9WW..."
...,...,...,...
493,girls,board games,"([, B092STXL3T, B09H7J66Z2, B00000JICB, B0BLHW..."
494,neutral,board games,"([, B092STXL3T, B07B7KPTQG, B078S8D27R, B078TW..."
495,boys,rock painting,"([, B08XK67TRQ, B07Z2R7S5P, B08RDCZWC7, B087PW..."
496,girls,rock painting,"([, B08XK67TRQ, B07Z2R7S5P, B08RDCZWC7, B087PW..."


In [25]:
# first20_link

In [26]:
len(first20.loc[0]['result'][1])

66

In [27]:
# first20 = pd.read_csv('item_results.csv')

In [28]:
first20.to_csv('az_uk_search_results.csv', index = False)
# first20_link.to_csv('item_links.csv', index = False)

In [51]:
df = pd.DataFrame.from_dict(search_result).T.reset_index()
df.rename(columns={'index':'item'}, inplace = True)
df

,item,boys,girls,neutral
0,toys,"(([, B0B9GL62T7, B0B8HZJZQF, B09X9SX2ZK, B0B42...","(([, B09CPH2Q1V, B085DGS9BN, B0B9BW3QGV, B087D...","(([, B09FT5P8RK, B09NLPL4TM, B09XBQGML5, B0B42..."
1,books,"(([, 1653075104, B09MDHH1CF, B0BJHF35GL, B09Y4...","(([, 1953424341, B0BLFSVS1G, B0B3RL89SB, 18485...","(([, B0BCCVT2S3, B0BBXZPG9J, 0241381223, 02415..."
2,learning material,"(([, B09N3S4D7N, B0BDCVF2X2, B0BHXQMQZB, B0BHX...","(([, B09N3S4D7N, B09SG8W7VS, B0BHXQMQZB, B0BHX...","(([, B08725XV15, B0B9VJYCZ7, B07M9D92SJ, B07MM..."
3,games,"(([, B0B91WFR9W, B07YRGFQHY, B088M5PZG8, B09L5...","(([, B09FTHMRGR, B0B3M432H5, 1687795347, B09MH...","(([, , B076PRWVFG, B09FTHMRGR, B0B91WFR9W, B08..."
4,sports,"(([, B09PYX62JJ, B08JZ25FFZ, B0BFF896D6, B08HG...","(([, B0BNVY8N37, B0B2JJMQ9J, B097GZD6WD, B0B4B...","(([, B09VBMYWK8, B093HB43VL, B078ZTYFWY, B09PY..."
